In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import numpy as np
import numpy as np
import os
import pandas as pd
import matplotlib.pylab as plt

from sklearn.utils import shuffle
import pandas as pd
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

META_SRC = "selected/0_0_0_metadata_extended.csv"
TRAIN_DATA_DIR = "selected"
TRAIN_X_LABEL = "uic_value"

VALID_META_SRC = "validation/selected/0_0_0_metadata_extended.csv"
VALID_DATA_DIR = "validation/selected"

train_meta_df = pd.read_csv(META_SRC)
train_meta_df = shuffle(train_meta_df)

valid_meta_df = pd.read_csv(VALID_META_SRC)
valid_meta_df = shuffle(valid_meta_df)


class ImgReader:
    def read(self, file: str, format_="jpg") -> np.ndarray:
        return plt.imread(file, format_)

    def read_multiple(self, files, format_="jpg") -> np.ndarray:
        return np.asarray([self.read(f, format_) for f in files])


img_reader = ImgReader()


def list_files_from_df(root_dir: str, df: pd.DataFrame):
    files = [os.path.join(root_dir, f) for f in df["filename"].values]

    return files


def get_all_images_with_column(data_dir, metadata_df, col_name: str, is_shuffle=False):
    metadata_df.dropna(inplace=True)
    metadata_df = metadata_df[metadata_df['uic_value'].str.len() == 15]
    df = metadata_df[["filename", col_name]]
    if is_shuffle:
        df = shuffle(df)
    files = list_files_from_df(data_dir, df)
    images = img_reader.read_multiple(files)
    col_values = df[col_name].values

    return images, col_values

def normalize_imgs(imgs):
    new_imgs_shape = (imgs.shape[0], imgs.shape[1], imgs.shape[2], 1)
    imgs = imgs.reshape(new_imgs_shape)
    imgs = imgs/255.0
    return imgs

is_shuffle = True
imgs, uids = get_all_images_with_column(TRAIN_DATA_DIR, train_meta_df, TRAIN_X_LABEL, is_shuffle)
imgs = normalize_imgs(imgs)

valid_imgs, valid_types = get_all_images_with_column(VALID_DATA_DIR, valid_meta_df, TRAIN_X_LABEL, is_shuffle)
valid_imgs = normalize_imgs(valid_imgs)


input_shape = (256, 320, 1)
valid_imgs.shape, imgs.shape

((125, 256, 320, 1), (74, 256, 320, 1))

In [9]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import numpy as np
import numpy as np
import os
import pandas as pd
import matplotlib.pylab as plt

from sklearn.utils import shuffle
import pandas as pd
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

META_SRC = "selected/0_0_0_metadata_extended.csv"
TRAIN_DATA_DIR = "selected"
TRAIN_X_LABEL = "uic_value"

VALID_META_SRC = "validation/selected/0_0_0_metadata_extended.csv"
VALID_DATA_DIR = "validation/selected"

train_meta_df = pd.read_csv(META_SRC)
train_meta_df = shuffle(train_meta_df)

valid_meta_df = pd.read_csv(VALID_META_SRC)
valid_meta_df = shuffle(valid_meta_df)


class ImgReader:
    def read(self, file: str, format_="jpg") -> np.ndarray:
        return plt.imread(file, format_)

    def read_multiple(self, files, format_="jpg") -> np.ndarray:
        return np.asarray([self.read(f, format_) for f in files])


img_reader = ImgReader()


def list_files_from_df(root_dir: str, df: pd.DataFrame):
    files = [os.path.join(root_dir, f) for f in df["filename"].values]

    return files


def get_all_images_with_column(data_dir, metadata_df, col_name: str, is_shuffle=False):
    metadata_df.dropna(inplace=True)
    metadata_df = metadata_df[metadata_df['uic_value'].str.len() == 15]
    df = metadata_df[["filename", col_name]]
    if is_shuffle:
        df = shuffle(df)
    files = list_files_from_df(data_dir, df)
    images = img_reader.read_multiple(files)
    col_values = df[col_name].values

    return images, col_values

def normalize_imgs(imgs):
    new_imgs_shape = (imgs.shape[0], imgs.shape[1], imgs.shape[2], 1)
    imgs = imgs.reshape(new_imgs_shape)
    imgs = imgs/255.0
    return imgs

is_shuffle = True
imgs, uids = get_all_images_with_column(TRAIN_DATA_DIR, train_meta_df, TRAIN_X_LABEL, is_shuffle)
imgs = normalize_imgs(imgs)

valid_imgs, valid_types = get_all_images_with_column(VALID_DATA_DIR, valid_meta_df, TRAIN_X_LABEL, is_shuffle)
valid_imgs = normalize_imgs(valid_imgs)


input_shape = (256, 320, 1)
valid_imgs.shape, imgs.shape

x_img = valid_imgs.reshape(-1, 256 * 320)
x_test = imgs.reshape(-1, 256 * 320)
tables = []
y_test_tables = []
for i in valid_types:
    tables.append([int(i) for i in i.replace('-', '').strip()[4:8]])
y = np.asarray(tables)
for i in uids:
    y_test_tables.append([int(i) for i in i.replace('-', '').strip()[4:8]])
y_test = np.asarray(y_test_tables)


class MLPNet(nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
        self.fc1 = nn.Linear(256 * 320, 3000)
        self.bn1 = nn.BatchNorm1d(num_features=3000)

        self.dropout = nn.Dropout(0.4)
        self.fc2 = nn.Linear(3000, 256)
        self.bn2 = nn.BatchNorm1d(num_features=256)

        self.fc33 = nn.Linear(256, 128)
        self.bn33 = nn.BatchNorm1d(num_features=128)

        self.fc333 = nn.Linear(128, 64)
        self.bn333 = nn.BatchNorm1d(num_features=64)

        self.fc3333 = nn.Linear(64, 128)
        self.bn3333 = nn.BatchNorm1d(num_features=128)

        self.dropout2 = nn.Dropout(0.4)
        self.fc3 = nn.Linear(128, 10)
        self.fc4 = nn.Linear(128, 10)
        self.fc5= nn.Linear(128, 10)

        self.fc6 = nn.Linear(128, 10)


    def forward(self, x):
        x = self.dropout(self.bn1(F.relu(self.fc1(x))))
        x = self.dropout2(self.bn2(F.relu(self.fc2(x))))
        x = self.dropout2(self.bn33(F.relu(self.fc33(x))))
        x = self.dropout2(self.bn333(F.relu(self.fc333(x))))
        x = self.dropout2(self.bn3333(F.relu(self.fc3333(x))))


        x2 = self.fc4(x)
        x3 = self.fc5(x)

        x4 = self.fc6(x)

        x = self.fc3(x)
        return F.log_softmax(x), F.log_softmax(x2), F.log_softmax(x3), F.log_softmax(x4)



model = MLPNet().to(device)

optimizer = optim.Adamax(model.parameters(), lr=6e-3)


def train(epoch=15):
    for i in range(epoch):
        model.train()
        data, target = torch.from_numpy(x_img).type(torch.cuda.FloatTensor), torch.from_numpy(np.asarray(y)).type(torch.cuda.FloatTensor)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        #         print(data.dtype, target.dtype)
        output1, output2, output3, output4 = model(data)
        loss1 = F.nll_loss(output1, torch.tensor(y[:, 0]).type(torch.cuda.LongTensor))
        loss2 = F.nll_loss(output2, torch.tensor(y[:, 1]).type(torch.cuda.LongTensor))
        loss3 = F.nll_loss(output3, torch.tensor(y[:, 2]).type(torch.cuda.LongTensor))

        loss4 = F.nll_loss(output4, torch.tensor(y[:, 3]).type(torch.cuda.LongTensor))

        loss = loss1 + loss2 + loss3 + loss4
        loss.backward()
        optimizer.step()
        print('Train Epoch: \tLoss: {:.6f}'.format(loss.item()))
        
        

        
def test(epoch=1):
    for i in range(epoch):
        model.eval()
        data, target = torch.from_numpy(x_test).float(), torch.from_numpy(np.asarray(y_test)).float()
        data, target = data.to(device), target.to(device)
        #         print(data.dtype, target.dtype)
        output1, output2, output3, output4 = model(data)
        loss1 = F.nll_loss(output1, torch.tensor(y_test[:, 0]).type(torch.cuda.LongTensor))
        loss2 = F.nll_loss(output2, torch.tensor(y_test[:, 1]).type(torch.cuda.LongTensor))
        loss3 = F.nll_loss(output3, torch.tensor(y_test[:, 2]).type(torch.cuda.LongTensor))

        loss4 = F.nll_loss(output4, torch.tensor(y_test[:, 3]).type(torch.cuda.LongTensor))
        loss = loss1 + loss2 + loss3 + loss4
        print(f'VAL {loss}')
        


def get_prob(data):
    model.eval()
    print(data.shape)
    if len(data.shape) <= 1:
        data = data.reshape(1, -1)
        print(data.shape)

    data = torch.from_numpy(data).float()
    data = data.to(device)
    output1, output2, output3, output4 = model(data)
    print('xd')
    return output1, output2, output3, output4



train()
test()
train()
get_prob(x_test[0])
test()
train()
test()
train()
test()
train()
test()
train()
test()
train()
test()
train()
test()
train()
test()
train()
test()
train()
test()
train()
test()
train()
test()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:133: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 	Loss: 9.512673
Train Epoch: 	Loss: 9.359974
Train Epoch: 	Loss: 9.306278
Train Epoch: 	Loss: 9.145170
Train Epoch: 	Loss: 8.855949
Train Epoch: 	Loss: 8.655087
Train Epoch: 	Loss: 8.363224
Train Epoch: 	Loss: 8.161484
Train Epoch: 	Loss: 8.025332
Train Epoch: 	Loss: 7.768181
Train Epoch: 	Loss: 7.487407
Train Epoch: 	Loss: 7.306524
Train Epoch: 	Loss: 7.245602
Train Epoch: 	Loss: 6.914216
Train Epoch: 	Loss: 6.771936
VAL 10.465245246887207
Train Epoch: 	Loss: 6.448531
Train Epoch: 	Loss: 6.286090
Train Epoch: 	Loss: 6.246289
Train Epoch: 	Loss: 6.030532
Train Epoch: 	Loss: 5.855522
Train Epoch: 	Loss: 5.637320
Train Epoch: 	Loss: 5.549393
Train Epoch: 	Loss: 5.392629
Train Epoch: 	Loss: 5.152772
Train Epoch: 	Loss: 4.925815
Train Epoch: 	Loss: 4.785442
Train Epoch: 	Loss: 4.623872
Train Epoch: 	Loss: 4.484368
Train Epoch: 	Loss: 4.429594
Train Epoch: 	Loss: 4.402902
(81920,)
(1, 81920)
xd
VAL 5.69647216796875
Train Epoch: 	Loss: 4.039452
Train Epoch: 	Loss: 3.925108
Train

## 